In [ ]:
import geopandas as gpd
from shapely.geometry import box
import re
import os

# === 1. 定义输入输出路径 === #
shp_folder = r'D:\file\a_clip_data\c_merge'
dsm_folder = r'D:\file\d_height'
output_file = r'D:\file\match_results.txt'  # 存储匹配结果的 txt 文件

shp_files = [f for f in os.listdir(shp_folder) if f.endswith('.shp')]
dsm_files = [f for f in os.listdir(dsm_folder) if f.endswith('_DEM.tif')]

# 提取经纬度的函数
pattern = re.compile(r'Copernicus_DSM_10_([NS])(\d+)_00_([EW])(\d+)_00_DEM')

def get_bbox_from_name(name):
    """从 DSM 文件名提取边界框"""
    match = pattern.search(name)
    if match:
        ns, lat, ew, lon = match.groups()
        lat = int(lat) * (1 if ns == 'N' else -1)
        lon = int(lon) * (1 if ew == 'E' else -1)
        return name, box(lon, lat, lon + 1, lat + 1)
    return None

# 创建 DSM 文件的边界框字典
dsm_bboxes = dict(filter(None, [get_bbox_from_name(f) for f in dsm_files]))

# === 2. 处理每个 shapefile 并写入结果文件 === #
with open(output_file, 'w', encoding='utf-8') as f_out:
    for shp_name in shp_files:
        shp_path = os.path.join(shp_folder, shp_name)

        # 读取 shapefile
        gdf = gpd.read_file(shp_path)

        # 过滤空 geometry
        gdf = gdf[gdf.geometry.notna()]
        gdf = gdf[~gdf.geometry.is_empty]

        # 计算 polygon 的 bbox
        gdf_bounds = gdf.geometry.bounds
        gdf['bbox_geom'] = gdf_bounds.apply(lambda row: box(row.minx, row.miny, row.maxx, row.maxy), axis=1)

        # === 3. 匹配 polygon bbox 和 DSM bbox === #
        gdf['intersecting_dsm_files'] = gdf['bbox_geom'].apply(
            lambda poly_box: [fname for fname, dsm_box in dsm_bboxes.items() if poly_box.intersects(dsm_box)]
        )

        # === 4. 将结果写入文件 === #
        #f_out.write(f"\n=== {shp_name} 匹配结果 ===\n")
        for idx, row in gdf.iterrows():
            matched_files = ", ".join(row['intersecting_dsm_files']) if row['intersecting_dsm_files'] else "无匹配"
            #f_out.write(f"Polygon {idx} 匹配到 DSM 文件: {matched_files}\n")
            f_out.write(f"{matched_files}\n")
            print(f"{matched_files}\n")

print(f"匹配结果已保存到 {output_file}")

Copernicus_DSM_10_N45_00_E004_00_DEM.tif, Copernicus_DSM_10_N45_00_E005_00_DEM.tif, Copernicus_DSM_10_N46_00_E004_00_DEM.tif, Copernicus_DSM_10_N46_00_E005_00_DEM.tif

Copernicus_DSM_10_N45_00_E015_00_DEM.tif, Copernicus_DSM_10_N45_00_E016_00_DEM.tif

Copernicus_DSM_10_N44_00_E033_00_DEM.tif, Copernicus_DSM_10_N44_00_E034_00_DEM.tif, Copernicus_DSM_10_N45_00_E033_00_DEM.tif, Copernicus_DSM_10_N45_00_E034_00_DEM.tif

Copernicus_DSM_10_N33_00_W078_00_DEM.tif, Copernicus_DSM_10_N33_00_W079_00_DEM.tif

Copernicus_DSM_10_N33_00_E130_00_DEM.tif, Copernicus_DSM_10_N33_00_E131_00_DEM.tif, Copernicus_DSM_10_N34_00_E130_00_DEM.tif, Copernicus_DSM_10_N34_00_E131_00_DEM.tif

Copernicus_DSM_10_N33_00_E132_00_DEM.tif

Copernicus_DSM_10_N33_00_W099_00_DEM.tif, Copernicus_DSM_10_N34_00_W099_00_DEM.tif

Copernicus_DSM_10_N33_00_W079_00_DEM.tif, Copernicus_DSM_10_N33_00_W080_00_DEM.tif, Copernicus_DSM_10_N34_00_W079_00_DEM.tif, Copernicus_DSM_10_N34_00_W080_00_DEM.tif

Copernicus_DSM_10_N33_00_E130_00_D